In [1]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
import numpy as np


In [ ]:
df = pd.read_csv('data.csv')

df['now'] = pd.Timestamp.now().date()

df['antiguedad'] = (pd.to_datetime(df.now, utc=True) -
                    pd.to_datetime(df.fecha_registro, utc=True)).dt.days
df.drop(['now', 'fecha_registro'], axis=1, inplace=True)

item_vars = ['id', 'title', 'condition', 'listing_type_id', 'buying_mode',
             'category_id', 'domain_id', 'price', 'original_price', 'sold_quantity',
             'available_quantity', 'accepts_mercadopago',
             'tags', 'logistic_type', 'nombre_garantia', 'n_garantia', 'precio', 'precio_base', 'cant_ini', 'unid_garantia', 'categoria']


In [ ]:
seller_vars = [el for el in df.columns if el not in item_vars]


cat_vars = [el for el in df.columns if df[el].dtype == 'O']
num_vars = [el for el in df.columns if el not in cat_vars]
ohe_vars = ['condition', 'listing_type_id', 'buying_mode',
            'domain_id', 'logistic_type', 'nombre_garantia', 'categoria']


In [ ]:
df.n_garantia.fillna(df.n_garantia.median(), inplace=True)
df.original_price.fillna(df.original_price.median(), inplace=True)
df.logistic_type.fillna(df.logistic_type.mode().values[0], inplace=True)
df.nombre_garantia.fillna(df.nombre_garantia.mode().values[0], inplace=True)


In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df[ohe_vars])
ohe_df = pd.DataFrame(enc.transform(
    df[ohe_vars]).toarray(), columns=enc.get_feature_names_out())

df = pd.concat([df, ohe_df], axis=1)


In [ ]:
df['n_garantia'] = np.where(df.unid_garantia == 'meses', df.n_garantia*30,
                            np.where(df.unid_garantia == 'años', df.n_garantia*365, df.n_garantia))
dt = df[seller_vars].drop_duplicates().dropna()


In [ ]:
# Medianas item_vars
med_df = df.groupby('id_seller')[['price', 'original_price',
                                  'available_quantity', 'n_garantia', 'precio', 'precio_base', 'sold_quantity', 'n_garantia']].median()
med_df.columns = [el+'_med' for el in med_df.columns]

dt = dt.merge(med_df, how='left', left_on='id_seller', right_index=True)


In [ ]:
# Maximos item_vars
max_df = df.groupby('id_seller')[['price', 'original_price',
                                  'available_quantity', 'n_garantia', 'precio', 'precio_base', 'sold_quantity', 'n_garantia']].max()
max_df.columns = [el+'_max' for el in max_df.columns]

dt = dt.merge(max_df, how='left', left_on='id_seller', right_index=True)


In [ ]:
# Minimos item_vars
min_df = df.groupby('id_seller')[['price', 'original_price',
                                  'available_quantity', 'n_garantia', 'precio', 'precio_base', 'sold_quantity', 'n_garantia']].min()
min_df.columns = [el+'_min' for el in min_df.columns]

dt = dt.merge(min_df, how='left', left_on='id_seller', right_index=True)


In [ ]:
# Cant. Vendidad
# NOTE: SE OBSERVO QUE MUCHOS SELLERS TIENEN LA MISMA CANTIDAD DE VENDIDOS, SE OBSERVA LA VARIABLE POR HISTOGRAMA
cant_df = df.groupby('id_seller')['sold_quantity'].sum()
"""fig = px.histogram(df, x='sold_quantity', histnorm='percent',
                   template='simple_white', nbins=50)
fig.show()
"""
dt = dt.merge(cant_df, how='left', left_on='id_seller', right_index=True)


In [ ]:
# Cant. Prod
prod_df = df.groupby('id_seller').title.nunique()
dt = dt.merge(prod_df, how='left', left_on='id_seller', right_index=True)


In [ ]:
# Ohe Vars Cant.
ohe_group = df.groupby('id_seller')[enc.get_feature_names_out()].sum()
dt = dt.merge(ohe_group, how='left', left_on='id_seller', right_index=True)


In [ ]:
dt.to_csv('clust.csv', index=False)
